In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
from linearmodels.panel import PanelOLS
from linearmodels.panel import compare
import datetime
import pickle

# 파일 불러오기
with open('../../data/df_da.pkl', 'rb') as f:
    df_da = pickle.load(f)


with open('../../data/df_postcnt_per_tag_year.pkl', 'rb') as f:
    df_postcnt_per_tag_year = pickle.load(f)

tag

In [ ]:
df_da['date']   = df_da['q_creationdate'].dt.date
df_da['year']   = df_da['q_creationdate'].dt.year
df_da['month']  = df_da['q_creationdate'].dt.month
df_da['day']    = df_da['q_creationdate'].dt.day

In [ ]:
df_da_q = df_da[['q_id', 'q_creationdate','q_tags']].drop_duplicates()

In [ ]:
df_da_q['year'] = df_da_q['q_creationdate'].dt.year
df_da_q['month'] = df_da_q['q_creationdate'].dt.month
df_da_q['day'] = df_da_q['q_creationdate'].dt.day
df_da_q['date'] = df_da_q['q_creationdate'].dt.date

In [ ]:
df_da_q['q_tags_list'] = df_da_q['q_tags'].apply(lambda x : x.strip().split(' '))

In [ ]:
print(df_da_q[['q_id']].shape)
print(df_da_q[['q_id']].drop_duplicates().shape)

In [ ]:
df_da_q = df_da_q.reset_index(drop=True)

In [ ]:
df_da_q_ex = df_da_q[['q_id', 'q_tags_list']].apply(pd.Series.explode)

In [ ]:
df_da_q_ex['q_tags_list'].value_counts().head(20)

In [ ]:
df_da_q_ex

In [ ]:
df_da_q['q_tagslen'] = df_da_q['q_tags'].apply(lambda x : len(x.strip().split(' ')))

In [ ]:
df_da_q_year = pd.merge(df_da_q[['q_id', 'year']], df_da_q_ex, on = 'q_id')

In [ ]:
df_da_q_year = df_da_q_year.groupby(['year','q_tags_list']).count()['q_id'].reset_index()
#  df_da.sort_values(by = ['date', 'q_id','qna_diff'], ascending=True).groupby(['date', 'q_id']).head(1) > 확인필요 

In [ ]:
df_da_q_year = df_da_q_year.sort_values(by = ['year', 'q_id'], ascending = False).groupby(['year']).head(10)

In [ ]:
df_da_q_year['year'] = df_da_q_year['year'].astype(str)

In [ ]:
# df_postcnt_per_tag_year

df_postcnt_per_tag_year_melt = df_postcnt_per_tag_year.melt(id_vars = ['year'], value_vars=df_postcnt_per_tag_year.columns[1:])

In [ ]:
# df_da_q_year.sort_values(by = ['year', 'q_id'], ascending = False).groupby(['year']).head(10)
df_postcnt_top10 = df_postcnt_per_tag_year_melt.sort_values(by = ['year', 'value'], ascending=False).groupby(['year']).head(10)

In [ ]:
df_postcnt_top10

In [ ]:
### create a figure with multiple subplots


sharey = True ## 또는 sharey=False
sharex = False ## 또는 sharex=False

fig, axs = plt.subplots(10, 2, figsize = (10, 40), sharey=sharey, sharex=sharex)

for x in range(0, 10):
        standard_year = 2014
        loop_year = str(standard_year + (x*1))
        print(loop_year)
    
        axs[x][0].bar(df_da_q_year[df_da_q_year['year'] ==loop_year].q_tags_list
                        ,  df_da_q_year[df_da_q_year['year'] ==loop_year].q_id
                        ,  color = 'skyblue'
                ) # hist type
        axs[x][0].set_title(str(loop_year), # add title
                pad = 10,
                loc = 'center')
        axs[x][0].tick_params(axis='x', rotation=45)

        axs[x][1].bar( df_postcnt_top10[df_postcnt_top10['year'] ==loop_year].variable
                ,  df_postcnt_top10[df_postcnt_top10['year'] ==loop_year].value
                ,  color = 'pink'
                ) # hist type
        axs[x][1].set_title(str(loop_year), # add title
                pad = 10,
                loc = 'center')
        axs[x][1].tick_params(axis='x', rotation=45)
 
# plt.suptitle("Top 10 Tags of Questions over 10 years")      
plt.tight_layout()    
plt.show();

In [ ]:
# 왼쪽이 tag를 1로 계산한 경우, 오른쪽이 질문별로 태그의 비중을 계산하여 1/n으로 계산한 경우 시각화 

# * 왼쪽 그래프와 오른쪽 그래프 사이의 갭이 클수록 곁다리 태그이다.(하나의 질문에 여러개의 태그가 들어가서 가중치가 작아지는 경우)
#     * 프론트에서는 자바스크립트가 아직도 절대적이기 때문에, 모든 프론트앤드 관련 질문에 javascript에 껴있을 수 있음 

# * ios, android의 경우 순위가 높아진 경우인데, 오히려 범용적이지 않기 때문에 순위가 높아졌다고 해석할 수 있음
# (java는 웹 앱 다 쓰인다고 하면 안드로이드는 특정 플랫폼만을 의미하므로)
# * c++도의 경우 웹프로그래밍이 불가능한 언어로 범용성이 떨어짐(기업에서 안씀) > 특정 임베디드와같은 목적성이 있는 곳에서 쓰임 

# * 2017년 이후 r등장 (왼쪽그래프에는 없으나 오른쪽 그래프에만 존재)

# * c#은 왜 많을까?(지운님께 물어보기) pos나 의료기기에서 C#많이씀

# * 2021년 부터 등장하는 pandas의 경우 python이랑 같이 나오므로 좌측에는 있으라 우측에는 없음 

# * reactjs는 자바스크립트의 하위라이브러리로 같이 묶여서 나올 가능성이 높음


